# Negative sentiment
* Anger
* Disgust
* Fear
* Sadness

# Positive sentiment
* Joy
* Trust

# Other sentiment
* Surprise
* Anticipation

In [1]:
import pandas as pd

In [2]:
psenti = pd.read_csv('dataset/market_dataset/psenti.csv')
resultsAAPL = pd.read_csv('dataset/market_dataset/resultsAAPL.csv')
resultsGoog = pd.read_csv('dataset/market_dataset/resultsGoog.csv')
resultsJPM = pd.read_csv('dataset/market_dataset/resultsJPM.csv')

t_c_1 = pd.read_csv('dataset/market_dataset/twitter_clean_text.001.csv')
column_names = t_c_1.columns

t_c_2 = pd.read_csv('dataset/market_dataset/twitter_clean_text.002.csv', names=column_names)
t_c_3 = pd.read_csv('dataset/market_dataset/twitter_clean_text.003.csv', names=column_names)

In [3]:
sentiment = pd.concat([psenti, resultsAAPL, resultsGoog, resultsJPM], ignore_index=True).drop_duplicates()
len(sentiment)

1225718

In [4]:
tweets = pd.concat([t_c_1, t_c_2, t_c_3], ignore_index=True).drop_duplicates()
len(tweets)

1225718

In [5]:
df = tweets.merge(sentiment, left_on='Id', right_on='Id')

In [6]:
df.to_csv('tweets.csv', index=False, sep=',', encoding='utf-8')

In [7]:
df = df.drop(['Id', 'Time', 'UserId', 'Company', 'Date', 'Original', 'Calculated', 'TotalSentimentWords', 'TotalWords'], axis=1)

In [46]:
df.head()

,Text,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,sentiment
0,putting apple's record-breaking quarter into c...,0,0,0,0,0,0,0,0,0
1,"""@wsj: apple plans to start shipping apple wat...",0,3,0,1,0,0,0,0,0
2,nike has tested the $92 level 3 times this yea...,0,1,0,0,0,0,1,1,0
3,"""@howardlindzon: me too (long) @sexonthebeach:...",0,1,0,0,0,0,0,0,0
4,@3score66 $c,0,0,0,0,0,0,0,0,0


In [77]:
all_adj = ['Joy', 'Trust', 'Anger', 'Disgust', 'Fear', 'Sadness']
pos_neg_adj = {
    1: all_adj[:2],
    -1: all_adj[2:],
}

def count_sent(row):
    tot = 0
    for a in all_adj:
        tot += row[a]
    return tot

def get_cont_sent(row):
    weighted_tot = 0
    for key, elem in pos_neg_adj.items():
        for a in elem:
            weighted_tot += key * row[a]
    tot_sent = count_sent(row)
    if tot_sent == 0:
        return 0
    else :
        return weighted_tot / tot_sent

In [90]:
cont_sent = []
for index, row in df.iterrows():
    cont_sent.append(get_cont_sent(row))

In [91]:
df['sentiment'] = cont_sent

In [96]:
df = df.drop(['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Trust'], axis=1)

KeyError: "['Anger' 'Anticipation' 'Disgust' 'Fear' 'Joy' 'Sadness' 'Surprise'\n 'Trust'] not found in axis"

In [97]:
df = df.rename(columns={"Text": "title"})

In [98]:
df.head()

,title,sentiment
0,putting apple's record-breaking quarter into c...,0.0
1,"""@wsj: apple plans to start shipping apple wat...",-1.0
2,nike has tested the $92 level 3 times this yea...,1.0
3,"""@howardlindzon: me too (long) @sexonthebeach:...",0.0
4,@3score66 $c,0.0


In [99]:
df.to_csv('dataset\market_dataset\market.csv', index=False, sep=',', encoding='utf-8')